In [1]:
import numpy as np
import torch
import pdb
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import struct
import gzip
from PIL import Image

def load_data_image():
    #读取文件
    binfile = open('../data/train-images.idx3-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前四个无符号整数
    magic, num, cows, cols = struct.unpack_from('>IIII',buffers,0)
    #获得偏移
    offset = struct.calcsize('>IIII')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num*cows*cols) + 'B'
    #读取图片数据并转化矩阵
    training_image = struct.unpack_from(strString, buffers, offset)
    training_image = np.reshape(training_image,(num,cows*cols))
    binfile.close()
    return (training_image)
def load_data_lable():
    #读取文件
    binfile = open('../data/train-labels.idx1-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前两个无符号整数
    magic, num = struct.unpack_from('>II',buffers,0)
    #获得偏移
    offset = struct.calcsize('>II')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num) + 'B'
    #读取图片数据并转化矩阵
    training_lable = struct.unpack_from(strString, buffers, offset)
    training_lable = np.reshape(training_lable,(num))
    binfile.close()
    return (training_lable)
def vectorized_result(j):
    e = np.zeros((10, 1)) 
    e[j] = 1.0 
    return e
training_image = load_data_image()
training_lable = load_data_lable()

test_image = training_image[50000:]
test_lable = training_lable[50000:]
training_image = [np.reshape(x, (784, 1)) for x in training_image]
training_lable = [vectorized_result(y) for y in training_lable]
test_image = [np.reshape(x, (784, 1)) for x in test_image]
training_image = training_image[:50000]
training_lable = training_lable[:50000]
training_data = zip(training_image, training_lable)
training_data = list(training_data)
test_data = zip(test_image,test_lable)
test_data = list(test_data)


In [10]:
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
def load(filename):
        
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    
    return net
# def sigmoid(x):
#     x_ravel = x.ravel()  # 将numpy数组展平
#     length = len(x_ravel)
#     y = []
#     for index in range(length):
#         if x_ravel[index] >= 0:
#             y.append(1.0 / (1 + np.exp(-x_ravel[index])))
#         else:
#             y.append(np.exp(x_ravel[index]) / (np.exp(x_ravel[index]) + 1))
#     return np.array(y).reshape(x.shape)
class CrossEntropyCost(object):
    @staticmethod 
    def fn(a, y): 
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a))) #交叉熵函数主体，用于尖酸loss
    @staticmethod 
    def delta(z, a, y): 
        return (a-y)         #交叉熵函数导数，用于反向传播
class QuadraticCost(object):
    @staticmethod 
    def fn(a, y): 
        return 0.5*np.linalg.norm(a-y)**2
    @staticmethod 
    def delta(z, a, y): 
        return (a-y) * sigmoid_prime(z)

class Network(object):
    def __init__(self,sizes, cost=CrossEntropyCost):
        self.num_layers = len(sizes) #sipzes神经网络规模例如输入[10,20,10]就代表第一层10个神经元，第二层20个神经元，第三层10个神经元
        self.sizes = sizes
        #新的初始化权重方法
        self.default_weight_initializer()
        self.cost = cost
    def default_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for  y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                       for x, y in zip(self.sizes[:-1], self.sizes[1:])]
    def large_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.rand(y, x)
                       for x, y in zip(self.sizes[:-1], self.sizes[1:])]
        
    def feedforward(self, a):       #前项计算
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)                          
        return a         
    def SGD(self, training_data, epochs, mini_batch_size, eta,
        lmbda = 0.0,
        evaluation_data = None,
        monitor_evaluation_cost = False,
        monitor_evaluation_accuracy= False,
        monitor_training_cost=False,
        monitor_training_accuracy=False):#mini_batch_size每次取样的规模None#eta 是学习速率，#epochs 迭代次数
        
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [],[]
        training_cost, training_accuracy = [], []
        for j in range(epochs): 
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta,lmbda,len(training_data))
            print("Epoch %s training complete" % j)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(
                self.accuracy(evaluation_data), n_data))
            print
        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                       for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                            for b, nb in zip(self.biases, nabla_b)]
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        delta = (self.cost).delta(zs[-1], activations[-1], y) #使用了交叉熵函数计算loss，这里是他的反向求导
        
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
            
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def accuracy(self, data, convert=False):
        
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                     for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                      for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)
    def total_cost(self, data, lmbda, convert=False):
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost
    def save(self, filename):
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.clost()

In [11]:
 net = Network([784,30,10])
training_cost, training_accuracy = net.SGD(training_data, 30, 10, 0.5,lmbda = 5.0,evaluation_data=test_data,monitor_evaluation_accuracy=True,monitor_evaluation_cost=True,monitor_training_accuracy=True,monitor_training_cost=True) 

H:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


Epoch 0 training complete
Cost on training data: 3.6723027269063495
Accuracy on training data: 5454 / 50000
Cost on evaluation data: 4.832637523710548
Accuracy on evaluation data: 1073 / 10000
Epoch 1 training complete
Cost on training data: 3.8287802475230523
Accuracy on training data: 7466 / 50000
Cost on evaluation data: 5.030336579275849
Accuracy on evaluation data: 1528 / 10000
Epoch 2 training complete
Cost on training data: 3.6266950061503684
Accuracy on training data: 9412 / 50000
Cost on evaluation data: 5.082655357080653
Accuracy on evaluation data: 1860 / 10000
Epoch 3 training complete
Cost on training data: 3.933028748981303
Accuracy on training data: 7354 / 50000
Cost on evaluation data: 4.9540532710988465
Accuracy on evaluation data: 1524 / 10000
Epoch 4 training complete
Cost on training data: 3.3456499201136465
Accuracy on training data: 10133 / 50000
Cost on evaluation data: 4.097767378011642
Accuracy on evaluation data: 1985 / 10000
Epoch 5 training complete
Cost on 

ValueError: too many values to unpack (expected 2)